In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd

# 分析经纬度格点上的各RCM输出结果
2021.12.14

用原始的还在添加的平均态绘制脚本，进行新添加模式的绘制

2021.12.20

添加到 CN05.1 mask后的数组上，只绘制中国区域；对边界ticks labels、色板、subplot进行调整，不再绘制空地图

**pre t2m 放置在同一个ipynb文件之中**

# PART1: DAILY PRECIPITATION

## 数据读入

目标处理：1991-2015年数据

In [ ]:
path_in = "/raid61/LS4P-TPEMIP/TEMP_DATA/precip_mask/"
ds_in = {}
dir_obs_in ="/raid61/LS4P-TPEMIP/REFERENCE/CN05.1/"
ds_in['obs'] = xr.open_dataset(dir_obs_in + "sel_CN05.1_Pre_1961_2018_daily_025x025.nc").rename({"pre":"pr"})
# ds_in[''] = xr.open_dataset(path_in + "")
ds_in['CWRF'] = xr.open_dataset(path_in + "CWRF_pre.nc") # 缺少逐日数据，只有多种尺度的平均态数据
ds_in['JAMSTECWRF'] = xr.open_dataset(path_in + "JAMSTECWRF_pre.nc") # 缺少4月
ds_in['NJU_LS4P_B'] = xr.open_dataset(path_in + "NJU_LS4P_B_pre.nc")
ds_in['NJU_LS4P_C'] = xr.open_dataset(path_in + "NJU_LS4P_C_pre.nc")
ds_in['NJU_Output-E'] = xr.open_dataset(path_in + "NJU_Output-E_pre.nc") # 缺少2015年
ds_in['SYS'] = xr.open_dataset(path_in + "SYS_pre.nc")
ds_in['Yang-ZhouX'] = xr.open_dataset(path_in + "Yang-ZhouX_pre.nc")
ds_in['YSU'] = xr.open_dataset(path_in + "YSU_pre.nc")

In [ ]:
# 稍微处理一下变量名，用于后续的处理
for single_mod in ds_in:
    print(single_mod)
    list_vars = list(ds_in[single_mod].variables)
    print(list_vars)
#     print(ds_in[single_mod].coords)
    if ('Time' in list_vars):
        print('----- rename Time>>>time')
        ds_in[single_mod] = ds_in[single_mod].rename({"Time":"time"})
    if ('pr' not in list_vars):
        print('------ rename ???pre???>>>pr')
        ds_in[single_mod] = ds_in[single_mod].rename({"dailypre":"pr"})
    if (ds_in[single_mod].time.shape[0] !=3825):
        print("**********time error in " + single_mod + " !" + " time length:" + str(ds_in[single_mod].time.shape[0]) )

## 平均态计算

In [ ]:
var_am = {}
var_jja= {}
for single_mod in ds_in:
    var_am[single_mod] = ds_in[single_mod] .sel(time = ds_in[single_mod].time.dt.month.isin([4,5])).mean('time') 
    var_jja[single_mod] = ds_in[single_mod] .sel(time = ds_in[single_mod].time.dt.month.isin([6,7,8])).mean('time')

## 绘制部分

In [ ]:
import matplotlib.colors as mcolors
cmap_data = [(1.0, 1.0, 1.0),
             (0.3137255012989044, 0.8156862854957581, 0.8156862854957581),
             (0.0, 1.0, 1.0),
             (0.0, 0.8784313797950745, 0.501960813999176),
             (0.0, 0.7529411911964417, 0.0),
             (0.501960813999176, 0.8784313797950745, 0.0),
             (1.0, 1.0, 0.0),
             (1.0, 0.6274510025978088, 0.0),
             (1.0, 0.0, 0.0),
             (1.0, 0.125490203499794, 0.501960813999176),
             (0.9411764740943909, 0.250980406999588, 1.0),
             (0.501960813999176, 0.125490203499794, 1.0),
             (0.250980406999588, 0.250980406999588, 1.0),
             (0.125490203499794, 0.125490203499794, 0.501960813999176),
             (0.125490203499794, 0.125490203499794, 0.125490203499794),
             (0.501960813999176, 0.501960813999176, 0.501960813999176),
             (0.8784313797950745, 0.8784313797950745, 0.8784313797950745),
             (0.9333333373069763, 0.8313725590705872, 0.7372549176216125),
             (0.8549019694328308, 0.6509804129600525, 0.47058823704719543),
             (0.6274510025978088, 0.42352941632270813, 0.23529411852359772),
             (0.4000000059604645, 0.20000000298023224, 0.0)]
ticks = np.arange(0,20,.5)
cmap = mcolors.ListedColormap(cmap_data, 'precipitation')
norm = mcolors.BoundaryNorm(ticks, cmap.N)

# 这个色板很适合降水，是NCL colortable中的 cmap_rmse = cmaps.MPL_s3pcpn_l

In [ ]:
# 国内政区图的绘制
# Load the border data, CN-border-La.dat is download from
# https://gmt-china.org/data/CN-border-La.dat
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.patches as mpatches
import cmaps
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

In [ ]:
# import proplot as plot
from matplotlib import pyplot as plt
import proplot as plot
# ----- get filter vars coords-----

# ticks = np.arange(0,22,1.)
#----- create plot -----
# fig, axs = plot.subplots(ncols=5, nrows=4, proj=('cyl') ) 
shape_array = [[1,2,3,4,5,],[0,6,7,8,9],[10,11,12,13,14],[0,15,16,17,18]]
fig, axs = plot.subplots(shape_array, proj=('cyl') )

#----- 添加海洋以及行政区划 -----
##---- 直接绘图，从边界文件添加
turn_on_border = True
if(turn_on_border):
    ##---- 使用shp文件添加
        ## shapefile数据下载的位置：
    coast_shapefile = "/home/yycheng/playground/shapefiles/gshhg-shp/GSHHS_shp/l/GSHHS_l_L1.shp"
    ninelines_shapefile     = "/home/yycheng/playground/shapefiles/SouthSea/" + "nine_lines.shp"
    ## 来源： 沛沛的诸省 + 诸岛
#     bou24p_shapefile     = "/m2data2/yycheng/data_stage/CN-border/peipeihelp/" + "bou2_4p.shp"
    ## 来源： https://www.resdc.cn/data.aspx?DATAID=200
    province_shapefile     = "/home/yycheng/playground/shapefiles/CN-sheng/" + "change_proj_CN-sheng-A.shp"

    for ax in axs:
        coast = shpreader.Reader(coast_shapefile).geometries()
        # world     = shpreader.Reader(world_border_shapefile).geometries()
        # river     = shpreader.Reader(river_border_shapefile).geometries()
#         river     = shpreader.Reader(river_border_shapefile, encoding = 'gbk')
        # bou24p    = shpreader.Reader(bou24p_shapefile).geometries()
        ninelines = shpreader.Reader(ninelines_shapefile).geometries()
        province  = shpreader.Reader(province_shapefile).geometries()
        ax.add_geometries(coast, ccrs.PlateCarree(), facecolor="none", edgecolor = 'k', linewidth=0.4, zorder=1 )
        # ax.add_geometries(world, ccrs.PlateCarree(), facecolor='none', edgecolor='k', linewidth=0.4, zorder=1)
        # ax.add_geometries(bou24p, ccrs.PlateCarree(), facecolor='none', edgecolor='k', linewidth=0.6, zorder=1) # 沛沛map
        ax.add_geometries(province, ccrs.PlateCarree(), facecolor='none', edgecolor='k', linewidth=0.6, zorder=1) # 地资所
        ax.add_geometries(ninelines, ccrs.PlateCarree(), facecolor='none', edgecolor='k', linewidth=0.6, zorder=1)
        # ax.add_geometries(river, ccrs.PlateCarree(), facecolor='none', edgecolor='b', linewidth=0.4, zorder=1)
        # 绘制部分的shapefile
#         for region in river.records():
#             if (region.attributes['NAME'] in ['黄河','长江']):
#                 # print("----- draw river! -----")
#                 # 此处需要使用 [] 让region.geometry可以迭代
#                 ax.add_geometries([region.geometry], ccrs.PlateCarree(), facecolor='none', edgecolor='b', linewidth=0.6, zorder=1)

axs[0:1,0].format(ltitle = "AM")
axs[2:3,0].format(ltitle = "JJA")

for ind,single_mod in enumerate(var_am):
        m_overlay = axs[ind].contourf(var_am[single_mod].lon.values, var_am[single_mod].lat.values, var_am[single_mod]['pr'].values, levels=ticks, cmap=cmap)
        axs[ind].format(ltitle = 'AM',title = single_mod)
for ind,single_mod in enumerate(var_jja):
        m_overlay = axs[ind+9].contourf(var_jja[single_mod].lon.values, var_jja[single_mod].lat.values, var_jja[single_mod]['pr'].values, levels=ticks, cmap=cmap)
        axs[ind+9].format(ltitle = 'JJA',title = single_mod)
fig.colorbar(m_overlay, loc='b', cmap=cmap, width=0.1, ticklabelsize=5,ticks=ticks[::2])

axs[0].format(title = 'CN05.1')
axs[9].format(title = 'CN05.1')
# ---------format作为整个proplot集中对属性进行修改的method---------
axs.format(
abc=True,
abcloc = 'ul',
#----- 地图底图设置 -----
# reso = 'x-hi',
reso = 'med',
# coast = False,
coast = False,
coastlinewidth = 0.4,
borders = False,
lakes = False,
land  = False,
ocean = False,
#-----GEO axis-----
lonlim=(70, 140), latlim=(15, 55),
labels = False, # 后续自己手动添加带线条的
gridlabelsize = 5,
gridminor = True,
lonlocator = np.arange(70,150,10),
latlocator = np.arange(15,55,10),
lonminorlocator = np.arange(70,140+2,2),
latminorlocator = np.arange(15,55+2,2),
longrid  = True,
latgrid  = True,
suptitle="precipitation [mm/d]",
)
for ax_ind in axs:
    ax_ind.set_xticks( plot.arange(70,140,10) ,crs=ccrs.PlateCarree()) # set longitude indicators
    ax_ind.set_yticks( plot.arange(15,55,10) , crs=ccrs.PlateCarree())
    lon_formatter = LongitudeFormatter(zero_direction_label=True,number_format='.0f')#,degree_symbol='')
    lat_formatter = LatitudeFormatter(number_format='.0f')#,degree_symbol='')
    ax_ind.xaxis.set_major_formatter(lon_formatter)
    ax_ind.yaxis.set_major_formatter(lat_formatter)
    
    ax_ind.tick_params(which='minor',direction='inout',length=4,width=1., top = False, right = False, labelsize = 7.)
    ax_ind.tick_params(which='major',direction='out', length=6, width=1.5 , colors='k', top = False, right = False, labelsize = 7.)

fig.patch.set_facecolor('white')
# fig.savefig('./output_pic/precip_spatial_mask_12.20.png', dpi=600, facecolor= "white")

In [ ]:
fig.savefig('./output_pic/precip_spatial_mask_12.20.png', dpi=600, facecolor= "white")

# PART2: DAILY MEAN surface air temperature

## 数据读入

目标处理：1991-2015年数据

In [ ]:
path_in = "/raid61/LS4P-TPEMIP/TEMP_DATA/t2m_mask/"
ds_in = {}
dir_obs_in ="/raid61/LS4P-TPEMIP/REFERENCE/CN05.1/"
ds_in['obs'] = xr.open_dataset(dir_obs_in + "sel_CN05.1_Tm_1961_2018_daily_025x025.nc").rename({'tm':'t2m'})
# ds_in[''] = xr.open_dataset(path_in + "")
ds_in['CWRF'] = xr.open_dataset(path_in + "CWRF_t2m.nc") # 缺少逐日数据，只有多种尺度的平均态数据
ds_in['JAMSTECWRF'] = xr.open_dataset(path_in + "JAMSTECWRF_t2m.nc") # 缺少4月
ds_in['NJU_LS4P_B'] = xr.open_dataset(path_in + "NJU_LS4P_B_t2m.nc")
ds_in['NJU_LS4P_C'] = xr.open_dataset(path_in + "NJU_LS4P_C_t2m.nc")
ds_in['NJU_Output-E'] = xr.open_dataset(path_in + "NJU_Output-E_t2m.nc") # 缺少2015年
ds_in['SYS'] = xr.open_dataset(path_in + "SYS_t2m.nc")
ds_in['Yang-ZhouX'] = xr.open_dataset(path_in + "Yang-ZhouX_t2m.nc")
ds_in['YSU'] = xr.open_dataset(path_in + "YSU_t2m.nc")

In [ ]:
# 稍微处理一下变量名，用于后续的处理
for single_mod in ds_in:
    print(single_mod)
    list_vars = list(ds_in[single_mod].variables)
    print(list_vars)
#     print(ds_in[single_mod].coords)
    if ('Time' in list_vars):
        print('----- rename Time>>>time')
        ds_in[single_mod] = ds_in[single_mod].rename({"Time":"time"})
#     if (ds_in[single_mod].time.shape[0] !=3825):
#         print("**********time error in " + single_mod + " !" + " time length:" + str(ds_in[single_mod].time.shape[0]) )

## 平均态计算

In [ ]:
var_am = {}
var_jja= {}
for single_mod in ds_in:
    var_am[single_mod] = ds_in[single_mod]['t2m'].sel(time = ds_in[single_mod].time.dt.month.isin([4,5])).mean('time') 
    var_jja[single_mod] = ds_in[single_mod]['t2m'].sel(time = ds_in[single_mod].time.dt.month.isin([6,7,8])).mean('time')

In [ ]:
for single_mod in ds_in:
    if(var_am[single_mod].max() > 200.):
        print("no!" + single_mod)
        var_am[single_mod] = var_am[single_mod] - 273.15
        var_jja[single_mod] = var_jja[single_mod] - 273.15

## 绘制部分

In [ ]:
import matplotlib.colors as mcolors
ticks = np.arange(-6,32,1)
# cmap=cmaps.GMT_wysiwygcont
cmap_diff = cmaps.cmp_flux 
diff_ticks = np.concatenate( ([-6],plot.arange(-4, 4, 0.25),[6]), axis = 0 )

In [ ]:
# 国内政区图的绘制
# Load the border data, CN-border-La.dat is download from
# https://gmt-china.org/data/CN-border-La.dat
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.patches as mpatches
import cmaps
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

In [ ]:
# import proplot as plot
from matplotlib import pyplot as plt
import proplot as plot
# ----- get filter vars coords-----

# ticks = np.arange(0,22,1.)
#----- create plot -----
# fig, axs = plot.subplots(ncols=5, nrows=4, proj=('cyl') ) 
shape_array = [[1,2,3,4,5,],[0,6,7,8,9],[10,11,12,13,14],[0,15,16,17,18]]
fig, axs = plot.subplots(shape_array, proj=('cyl') )

#----- 添加海洋以及行政区划 -----
##---- 直接绘图，从边界文件添加
turn_on_border = True
if(turn_on_border):
    ##---- 使用shp文件添加
        ## shapefile数据下载的位置：
    coast_shapefile = "/home/yycheng/playground/shapefiles/gshhg-shp/GSHHS_shp/l/GSHHS_l_L1.shp"
    ninelines_shapefile     = "/home/yycheng/playground/shapefiles/SouthSea/" + "nine_lines.shp"
    ## 来源： 沛沛的诸省 + 诸岛
#     bou24p_shapefile     = "/m2data2/yycheng/data_stage/CN-border/peipeihelp/" + "bou2_4p.shp"
    ## 来源： https://www.resdc.cn/data.aspx?DATAID=200
    province_shapefile     = "/home/yycheng/playground/shapefiles/CN-sheng/" + "change_proj_CN-sheng-A.shp"

    for ax in axs:
        coast = shpreader.Reader(coast_shapefile).geometries()
        # world     = shpreader.Reader(world_border_shapefile).geometries()
        # river     = shpreader.Reader(river_border_shapefile).geometries()
#         river     = shpreader.Reader(river_border_shapefile, encoding = 'gbk')
        # bou24p    = shpreader.Reader(bou24p_shapefile).geometries()
        ninelines = shpreader.Reader(ninelines_shapefile).geometries()
        province  = shpreader.Reader(province_shapefile).geometries()
        ax.add_geometries(coast, ccrs.PlateCarree(), facecolor="none", edgecolor = 'k', linewidth=0.4, zorder=1 )
        # ax.add_geometries(world, ccrs.PlateCarree(), facecolor='none', edgecolor='k', linewidth=0.4, zorder=1)
        # ax.add_geometries(bou24p, ccrs.PlateCarree(), facecolor='none', edgecolor='k', linewidth=0.6, zorder=1) # 沛沛map
        ax.add_geometries(province, ccrs.PlateCarree(), facecolor='none', edgecolor='k', linewidth=0.6, zorder=1) # 地资所
        ax.add_geometries(ninelines, ccrs.PlateCarree(), facecolor='none', edgecolor='k', linewidth=0.6, zorder=1)
        # ax.add_geometries(river, ccrs.PlateCarree(), facecolor='none', edgecolor='b', linewidth=0.4, zorder=1)
        # 绘制部分的shapefile
#         for region in river.records():
#             if (region.attributes['NAME'] in ['黄河','长江']):
#                 # print("----- draw river! -----")
#                 # 此处需要使用 [] 让region.geometry可以迭代
#                 ax.add_geometries([region.geometry], ccrs.PlateCarree(), facecolor='none', edgecolor='b', linewidth=0.6, zorder=1)

axs[0:1,0].format(ltitle = "AM")
axs[2:3,0].format(ltitle = "JJA")

for ind,single_mod in enumerate(var_am):
        if (single_mod == 'obs'):
            m_overlay = axs[ind].contourf(var_am[single_mod].lon.values, var_am[single_mod].lat.values, var_am[single_mod].values, levels=ticks, cmap=cmap)
        if (single_mod != 'obs'):
            m_diff = axs[ind].contourf(var_am[single_mod].lon.values, var_am[single_mod].lat.values, var_am[single_mod].values - var_am['obs'].values\
                                       , levels=diff_ticks, cmap=cmap_diff, norm = "midpoint")
        axs[ind].format(ltitle = 'AM',title = single_mod)
        
for ind,single_mod in enumerate(var_jja):
#         m_overlay = axs[ind+9].contourf(var_jja[single_mod].lon.values, var_jja[single_mod].lat.values, var_jja[single_mod].values, levels=ticks, cmap=cmap)
        if (single_mod == 'obs'):
            m_overlay = axs[ind+9].contourf(var_jja[single_mod].lon.values, var_jja[single_mod].lat.values, var_jja[single_mod].values, levels=ticks, cmap=cmap)
        if (single_mod != 'obs'):
            m_diff = axs[ind+9].contourf(var_jja[single_mod].lon.values, var_jja[single_mod].lat.values, var_jja[single_mod].values - var_jja['obs'].values\
                                       , levels=diff_ticks, cmap=cmap_diff, norm = "midpoint")
        axs[ind+9].format(ltitle = 'JJA',title = single_mod)

fig.colorbar(m_overlay, loc='b', cmap=cmap, width=0.1, ticklabelsize=5,ticks=ticks[::2] ,title='temperature[' + r'$^\circ C$' +']')
fig.colorbar(m_diff, loc='b', cmap=cmap_diff, width=0.1, ticklabelsize=5,ticks=diff_ticks[:] ,title='temperature bias[' + r'$^\circ C$' +']')

axs[0].format(title = 'CN05.1')
axs[9].format(title = 'CN05.1')
# ---------format作为整个proplot集中对属性进行修改的method---------
axs.format(
abc=True,
abcloc = 'ul',
#----- 地图底图设置 -----
# reso = 'x-hi',
reso = 'med',
# coast = False,
coast = False,
coastlinewidth = 0.4,
borders = False,
lakes = False,
land  = False,
ocean = False,
#-----GEO axis-----
lonlim=(70, 140), latlim=(15, 55),
labels = False, # 后续自己手动添加带线条的
gridlabelsize = 5,
gridminor = True,
lonlocator = np.arange(70,150,10),
latlocator = np.arange(15,55,10),
lonminorlocator = np.arange(70,140+2,2),
latminorlocator = np.arange(15,55+2,2),
longrid  = True,
latgrid  = True,
suptitle="surface air temperature [" +r"$^\circ C$"+ "]",
)
for ax_ind in axs:
    ax_ind.set_xticks( plot.arange(70,140,10) ,crs=ccrs.PlateCarree()) # set longitude indicators
    ax_ind.set_yticks( plot.arange(15,55,10) , crs=ccrs.PlateCarree())
    lon_formatter = LongitudeFormatter(zero_direction_label=True,number_format='.0f')#,degree_symbol='')
    lat_formatter = LatitudeFormatter(number_format='.0f')#,degree_symbol='')
    ax_ind.xaxis.set_major_formatter(lon_formatter)
    ax_ind.yaxis.set_major_formatter(lat_formatter)
    
    ax_ind.tick_params(which='minor',direction='inout',length=4,width=1., top = False, right = False, labelsize = 7.)
    ax_ind.tick_params(which='major',direction='out', length=6, width=1.5 , colors='k', top = False, right = False, labelsize = 7.)

fig.patch.set_facecolor('white')
# fig.savefig('./output_pic/precip_spatial_mask_12.20.png', dpi=600, facecolor= "white")

In [ ]:
fig.savefig('./output_pic/t2m_spatial_mask_12.20.png', dpi=600, facecolor= "white")